In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10 as cf10

In [6]:
(train_images, train_labels), (test_images, test_labels) = cf10.load_data()

In [7]:
# Reduce dataset size to 1/5 of the original to avoid memory overload during image resizing
train_images, train_labels = train_images[:10000], train_labels[:10000]
test_images, test_labels = test_images[:1000], test_labels[:1000]

In [8]:
# Flatten the labels arrays to remove the extra dimension
train_labels = train_labels.flatten()
test_labels = test_labels.flatten()

In [9]:
# Check shapes before resizing
print("Original Shapes:")
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

Original Shapes:
(10000, 32, 32, 3)
(10000,)
(1000, 32, 32, 3)
(1000,)


In [10]:
# pad the images
train_images = np.pad(train_images, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')
test_images = np.pad(test_images, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')

In [11]:
# Check the new shapes after padding
print("Upload Image Shapes:")
print(f"Training set: {train_images.shape}")
print(f"Test set: {test_images.shape}")

Upload Image Shapes:
Training set: (10000, 36, 36, 3)
Test set: (1000, 36, 36, 3)


In [12]:
# One-hot encode the labels
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
train_labels = to_categorical(train_labels, len(class_names))
test_labels = to_categorical(test_labels, len(class_names))

In [13]:
# Normalize image data: pixel values scaled between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [14]:
# Define a function to resize images to 227x227
def resize_images(images):
  return tf.image.resize(images, [227, 227])

# Resize both train and test images
train_images = resize_images(train_images)
test_images = resize_images(test_images)

# Check shapes  after resizing
print("\nResized Shapes:")
print(train_images.shape)
print(test_images.shape)


Resized Shapes:
(10000, 227, 227, 3)
(1000, 227, 227, 3)


In [15]:
# Verify the dataset after preprocessing
print(f"Training Data Shape: {train_images.shape}, Training Labels Shape: {train_labels.shape}")
print(f"Test Data Shape: {test_images.shape}, Test Labels Shape: {test_labels.shape}")

Training Data Shape: (10000, 227, 227, 3), Training Labels Shape: (10000, 10)
Test Data Shape: (1000, 227, 227, 3), Test Labels Shape: (1000, 10)


In [16]:
# Variables for AlexNet architecture
INPUT_SHAPE =(227, 227, 3)
FILTER1_SIZE = 96
FILTER2_SIZE = 256
FILTER3_SIZE = 384
FILTER4_SIZE = 384
FILTER5_SIZE = 256
FILTER_SHAPE1 = (11, 11)
FILTER_SHAPE2 = (5, 5)
FILTER_SHAPE3 = (3, 3)
POOL_SHAPE = (3, 3)
POOL_STRIDE = (2, 2)
FULLY_CONNECT_NUM1 = 4096
FULLY_CONNECT_NUM2 = 4096
NUM_CLASSES = len(class_names)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 27, 27, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 13, 13, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 6, 6, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │        40,970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,324,746 (222.49 MB)

 Trainable params: 58,323,530 (222.49 MB)

 Non-trainable params: 1,216 (4.75 KB)

In [ ]:
from tensorflow.keras import metrics

BATCH_SIZE = 32
EPOCHS = 50

METRICS = ['accuracy',
           metrics.Precision(name='precision'),
           metrics.Recall(name = 'recall')]

model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = METRICS)

#Train the model
training_history = model.fit(train_images, train_labels,
                             batch_size = BATCH_SIZE,
                             epochs = EPOCHS,
                             validation_data = (test_images, test_labels),verbose=1)

Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1802s 6s/step - accuracy: 0.2178 - loss: 3.5822 - precision: 0.2649 - recall: 0.1031 - val_accuracy: 0.2200 - val_loss: 2.8305 - val_precision: 0.4484 - val_recall: 0.1390
Epoch 2/50
 10/313 ━━━━━━━━━━━━━━━━━━━━ 26:28 5s/step - accuracy: 0.3163 - loss: 1.8786 - precision: 0.4543 - recall: 0.1529